<a href="https://colab.research.google.com/github/Adler-1014/SNU_ML_GROUP2/blob/developSrk/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path='/content/drive/MyDrive/머신러닝과제/Train.csv'

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv(file_path)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [6]:
# statistical info
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
!pip install pycaret

In [8]:
train = df.sample(frac=0.80, random_state=786)
test = df.drop(train.index)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [11]:
train.shape[0]

6818

In [12]:
test.shape[0]

1705

In [17]:
from pycaret.regression import * #변경 필요시 : pycaret.classification

exp_clf = setup(data = train, target = 'Item_Outlet_Sales', session_id=786)

,Description,Value
0,Session id,786
1,Target,Item_Outlet_Sales
2,Target type,Regression
3,Original data shape,"(6818, 12)"
4,Transformed data shape,"(6818, 47)"
5,Transformed train set shape,"(4772, 47)"
6,Transformed test set shape,"(2046, 47)"
7,Numeric features,4
8,Categorical features,7
9,Rows with missing values,45.2%


In [18]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,895.9029,1652877.9255,1285.0061,0.4327,0.6239,0.6562,1.0050
lightgbm,Light Gradient Boosting Machine,899.0079,1714780.5162,1308.8803,0.4115,0.6263,0.6388,1.4930
rf,Random Forest Regressor,908.8465,1733380.9262,1316.1900,0.4050,0.6243,0.6270,4.1490
lr,Linear Regression,977.5227,1772285.7455,1330.7306,0.3922,0.7960,1.0852,1.0720
ridge,Ridge Regression,977.6583,1772899.0003,1330.9597,0.3919,0.7959,1.0850,0.2190
lasso,Lasso Regression,978.0484,1775117.5209,1331.7737,0.3912,0.7911,1.0824,0.2420
llar,Lasso Least Angle Regression,978.0201,1775087.0192,1331.7627,0.3912,0.7909,1.0823,0.2000
br,Bayesian Ridge,979.5032,1781037.2035,1333.9989,0.3892,0.7921,1.0827,0.2010
et,Extra Trees Regressor,921.9789,1780672.2780,1333.6628,0.3890,0.6269,0.6302,3.4410
xgboost,Extreme Gradient Boosting,935.7176,1826493.2250,1350.8634,0.3730,0.6896,0.6623,0.5770


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='squared_error',
                          max_depth=3, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_samples_leaf=1,
                          min_samples_split=2, min_weight_fraction_leaf=0.0,
                          n_estimators=100, n_iter_no_change=None,
                          random_state=786, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
gbr = create_model('gbr') #gbr 그래디언트 부스팅 객체 생성

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,862.1452,1660732.8146,1288.6942,0.5103,0.5953,0.6435
1,910.0368,1636631.3345,1279.3089,0.4332,0.5795,0.5825
2,869.6938,1487263.8919,1219.5343,0.4830,0.6306,0.6864
3,970.0610,1768197.9827,1329.7361,0.3853,0.6119,0.6193
4,894.5221,1631219.4908,1277.1920,0.3890,0.6242,0.6485
5,914.5689,1850519.3653,1360.3380,0.4283,0.7000,0.7844
6,888.7048,1688886.5297,1299.5717,0.3920,0.6352,0.6563
7,913.3964,1666697.0705,1291.0062,0.4198,0.6132,0.6184
8,870.0366,1646957.4592,1283.3384,0.4125,0.6303,0.6649


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
rf = create_model('rf') #rf 랜덤포레스트 객체 생성

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,881.1702,1731930.4946,1316.0283,0.4893,0.5980,0.6266
1,932.5573,1737484.3933,1318.1367,0.3982,0.5902,0.5743
2,898.5025,1657905.6859,1287.5969,0.4237,0.6258,0.6546
3,961.3993,1772124.8114,1331.2118,0.3839,0.6185,0.5840
4,892.1874,1600874.8462,1265.2568,0.4004,0.6060,0.6243
5,904.6982,1886505.8397,1373.5013,0.4171,0.6752,0.7229
6,914.0751,1821854.1812,1349.7608,0.3441,0.6478,0.6386
7,923.4555,1762986.5119,1327.7750,0.3863,0.6324,0.6030
8,890.1415,1746460.4871,1321.5372,0.3770,0.6253,0.6184


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
tune_model(estimator = gbr, optimize='R2') #모델 튜닝

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,828.9420,1521550.1156,1233.5113,0.5513,0.6220,0.6509
1,842.3509,1414049.8228,1189.1383,0.5103,0.6159,0.5689
2,836.3702,1357053.2465,1164.9263,0.5283,0.6253,0.7097
3,898.4255,1530330.9083,1237.0654,0.4680,0.5942,0.6304
4,842.8260,1398742.8276,1182.6846,0.4761,0.6041,0.6537
5,849.5421,1588485.5783,1260.3514,0.5092,0.6552,0.7443
6,811.4778,1368130.7528,1169.6712,0.5075,0.5958,0.6396
7,878.3431,1489451.4291,1220.4308,0.4815,0.6393,0.6517
8,825.7513,1405327.9456,1185.4653,0.4987,0.6366,0.6730


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='squared_error',
                          max_depth=5, max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.05, min_samples_leaf=1,
                          min_samples_split=7, min_weight_fraction_leaf=0.0,
                          n_estimators=50, n_iter_no_change=None,
                          random_state=786, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
tune_model(estimator = rf, optimize='R2') #모델 튜닝

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:42:17
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [28]:
blend_best2 = blend_models(estimator_list=[gbr, rf], fold=5, optimize='R2')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,896.4624,1681496.1687,1296.7252,0.4660,0.5994,0.6282
1,955.9356,1795315.6381,1339.8939,0.3764,0.6314,0.6510
2,924.6064,1817268.3367,1348.0610,0.3846,0.6639,0.7216
3,918.5600,1806707.1790,1344.1381,0.3614,0.6409,0.6444
4,901.4463,1704165.5851,1305.4369,0.3955,0.6491,0.6911
Mean,919.4022,1760990.5815,1326.8510,0.3968,0.6369,0.6673
Std,21.0292,56539.9760,21.3773,0.0363,0.0216,0.0341


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
pred = predict_model(gbm)

In [ ]:
final_model = finalize_model(gbm)
prep_pipe = get_config('prep_pipe')
prep_pipe.steps.append(['trained_model', final_model])
pred = prep_pipe.predict_proba(test)

# final_model = finalize_model(blend_best2)
# pred = predict_model(final_model, data=test)